I want to know if we have access to user partisanship for each topic.

In [1]:
import pandas as pd

In [2]:
# check note_info file

note_info = pd.read_parquet('aux_note_info.parquet', engine='auto')

In [3]:
len(note_info)

1946619

In [4]:
note_info.columns

Index(['noteId', 'ratingWeight', 'createdAtMillis', 'noteAuthorParticipantId',
       'awaitingBool', 'numRatingsLast28', 'currentStatus', 'crhBool',
       'crnhBool', 'unlockedRatingStatus', 'helpfulOther',
       'helpfulInformative', 'helpfulClear', 'helpfulEmpathetic',
       'helpfulGoodSources', 'helpfulUniqueContext', 'helpfulAddressesClaim',
       'helpfulImportantContext', 'helpfulUnbiasedLanguage', 'notHelpfulOther',
       'notHelpfulIncorrect', 'notHelpfulSourcesMissingOrUnreliable',
       'notHelpfulOpinionSpeculationOrBias', 'notHelpfulMissingKeyPoints',
       'notHelpfulOutdated', 'notHelpfulHardToUnderstand',
       'notHelpfulArgumentativeOrBiased', 'notHelpfulOffTopic',
       'notHelpfulSpamHarassmentOrAbuse', 'notHelpfulIrrelevantSources',
       'notHelpfulOpinionSpeculation', 'notHelpfulNoteNotNeeded',
       'notHelpfulOtherAdjusted', 'notHelpfulIncorrectAdjusted',
       'notHelpfulSourcesMissingOrUnreliableAdjusted',
       'notHelpfulOpinionSpeculationOrBi

In [5]:
# check helpfulness scores

h_scores = pd.read_parquet('helpfulness_scores.parquet', engine='auto')

In [6]:
h_scores.columns

Index(['raterParticipantId', 'coreRaterIntercept', 'coreRaterFactor1',
       'crhCrnhRatioDifference', 'meanNoteScore', 'raterAgreeRatio',
       'successfulRatingHelpfulCount', 'successfulRatingNotHelpfulCount',
       'successfulRatingTotal', 'unsuccessfulRatingHelpfulCount',
       'unsuccessfulRatingNotHelpfulCount', 'unsuccessfulRatingTotal',
       'ratingsAwaitingMoreRatings', 'ratedAfterDecision',
       'notesCurrentlyRatedHelpful', 'notesCurrentlyRatedNotHelpful',
       'notesAwaitingMoreRatings', 'enrollmentState',
       'successfulRatingNeededToEarnIn', 'authorTopNotHelpfulTagValues',
       'timestampOfLastStateChange', 'aboveHelpfulnessThreshold',
       'isEmergingWriter', 'aggregateRatingReceivedTotal',
       'timestampOfLastEarnOut', 'groupRaterIntercept', 'groupRaterFactor1',
       'modelingGroup', 'raterHelpfulnessReputation', 'numberOfTimesEarnedOut',
       'expansionRaterIntercept', 'expansionRaterFactor1',
       'expansionPlusRaterIntercept', 'expansionPlus

In [2]:
# load rating file
chunk_size = 100000
monthly_count = dict()
# with open('ratings/ratings-00000.tsv', 'r') as r:
#     ratings = pd.read_csv(r, sep='\t', chunksize=chunk_size)
with pd.read_csv('ratings/ratings-00000.tsv', sep='\t', chunksize=chunk_size) as reader:
    for idx, chunk in enumerate(reader):
        chunk['createdAt'] = pd.to_datetime(chunk['createdAtMillis'], unit='ms')
        monthly_count[idx] = chunk.groupby([chunk['createdAt'].dt.year,
                                           chunk['createdAt'].dt.month])['noteId']\
                                            .count()
        # print(type(chunk))

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [62]:
sample_1 = grouped.sample(10)
sample_2 = grouped.sample(5)

In [80]:
sample_1

,year,month,raterParticipantId
0,2021,10,2
3,2022,5,1
38,2025,4,342
1,2021,11,2
2,2022,4,3
26,2024,4,317
41,2025,7,349
35,2025,1,479
8,2022,10,17
27,2024,5,353


In [68]:
merged = pd.merge(sample_2, sample_1, on=['year', 'month'], how='outer')

In [70]:
merged.fillna(0, inplace=True)

In [84]:
# this will hold the cumulative grouped result
final_result = pd.DataFrame(columns=["year", "month", "unique_id_count"])
df1 = chunk.sample(5000)
df2 = chunk.sample(5000)
# Example: imagine you process your dataframes one by one
for df in [df1, df2]:  # in real code you load/process one df at a time
    # group the current dataframe
    grouped = df.groupby(["year", "month"])["raterParticipantId"].nunique().reset_index(name="unique_id_count")
    # merge with the running final_result
    final_result = pd.merge(
        final_result, grouped,
        on=["year", "month"], how="outer", suffixes=("", "_new")
    )

    # sum counts (fillna for missing)
    final_result["unique_id_count"] = (
        final_result["unique_id_count"].fillna(0)
        + final_result["unique_id_count_new"].fillna(0)
    )

    # drop the temporary column
    final_result = final_result.drop(columns=["unique_id_count_new"])

/tmp/ipykernel_4951/1539195444.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_result["unique_id_count"].fillna(0)


In [85]:
df1.groupby(["year", "month"])["raterParticipantId"].nunique().reset_index(name="unique_id_count")

,year,month,unique_id_count
0,2022,6,1
1,2022,10,4
2,2022,11,3
3,2022,12,7
4,2023,1,8
5,2023,2,12
6,2023,3,13
7,2023,4,19
8,2023,5,23
9,2023,6,27


In [86]:
df2.groupby(["year", "month"])["raterParticipantId"].nunique().reset_index(name="unique_id_count")

,year,month,unique_id_count
0,2021,11,1
1,2022,7,1
2,2022,9,1
3,2022,10,1
4,2022,11,5
5,2022,12,5
6,2023,1,8
7,2023,2,3
8,2023,3,9
9,2023,4,21


In [87]:
final_result

,year,month,unique_id_count
0,2021,11,1.0
1,2022,6,1.0
2,2022,7,1.0
3,2022,9,1.0
4,2022,10,5.0
5,2022,11,8.0
6,2022,12,12.0
7,2023,1,16.0
8,2023,2,15.0
9,2023,3,22.0
